In [3]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algera
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from torch import FloatTensor
import time
import matplotlib.pyplot as plt


from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, BatchNormalization,  Dropout , Activation
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.losses import BinaryCrossentropy


def build_prediction_model():

    model = Sequential()
        
    model.add(Dense(2048, input_dim = 785))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(32))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy',#[binary_focal_loss(alpha=.25, gamma=2)],
                              optimizer='adam',
                              metrics=[AUC(), 'accuracy', Precision(), Recall()])
#     print(model.summary())
    return model


basepath = "../input/siim-isic-melanoma-classification/"

x,y = 256, 256

batch_size = 16
valid_size = 16

seg_gen = ImageDataGenerator()

def normalize_df(filename = None, type = 'test', df = None ):
    if  filename != None:
        df = pd.read_csv(filename)
    if not list(df['image_name'].astype(str))[0].endswith('.jpg'):
        df['image_name']+='.jpg' 
    #one hot encoding of anatomical features
    columns = ['image_name','sex', 'age_approx', 'anatom_site_general_challenge']
    if type != 'test':
        columns = ['target']+columns
        df['target'] = df['target'].astype(str)
    df = df[columns]
    df['sex'].replace({'male':1, 'female':0}, inplace = True)
    df['anatom_site_general_challenge'].replace({np.nan:'nan'}, inplace = True)
    anatomic_sites = ['upper extremity', 'oral/genital', 'palms/soles', 'torso', 'head/neck', 'lower extremity', 'nan']
    for btype in anatomic_sites:
        df[btype] = df['anatom_site_general_challenge'] == btype
        df[btype] = df[btype].astype(int)
    df.drop(['anatom_site_general_challenge'], inplace = True, axis = 1)
    #z-transform age
    df['age_approx'] = (df['age_approx']-np.mean(df['age_approx']))/np.std(df['age_approx'])
    df.dropna(inplace = True)
    return df.sample(len(df)) #shuffle dataframe

test_info = normalize_df(basepath + 'test.csv')
seg_info = normalize_df(basepath + "train.csv", type = 'train')


#undersample majority class
malignant  = seg_info[seg_info['target'] == '1']
benign = seg_info[seg_info['target'] == '0']
benign = benign.sample(len(malignant))

seg_info = pd.concat([benign, malignant])

skf = StratifiedKFold(n_splits=5) # split 80/20 into test/valid data
for train_index,  val_index in skf.split(np.zeros(len(seg_info)), seg_info['target']):
    break
del skf

#shuffle dataframes
train_df = seg_info.iloc[train_index].sample(len(train_index))
val_df = seg_info.iloc[val_index].sample(len(val_index))

print(val_df['target'].value_counts())
print(train_df['target'].value_counts())

seg_train_generator = seg_gen.flow_from_dataframe(
    dataframe = train_df,
    directory = basepath + 'jpeg/train/',
    x_col = 'image_name',
    y_col = 'target',
    target_size =(x, y), # resize images
    batch_size = batch_size,
    validate_filenames = False,
    shuffle = False,#preserve order in df to access demographic info
    class_mode = 'binary'
)

seg_valid_generator = seg_gen.flow_from_dataframe(
    dataframe = val_df,
    directory = basepath + 'jpeg/train/',
    x_col = 'image_name',
    y_col = 'target',
    target_size =(x, y),
    batch_size = valid_size,
    validate_filenames = False,
    shuffle = False,
    class_mode = 'binary'
)

seg_test_generator = seg_gen.flow_from_dataframe(
    dataframe = test_info,
    directory = basepath + 'jpeg/test/',
    x_col = 'image_name',
    y_col = None,
    target_size =(x, y),
    batch_size = 17,
    validate_filenames = False,
    shuffle = False,
    class_mode = None
)

segmentation_model = load_model('../input/bdcunet/weight_isic18.hdf5')
prediction_model = build_prediction_model()

def rotate_image(image, angle, image_center):
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def get_asymmetry_from_contours(img, big_contours):
    major_axis_lengths = []
    minor_axis_lengths = []
    asymmetry_vs = []
    asymmetry_hs = []
    for idx, bc in enumerate(big_contours): #get asymmetry metrics for each sufficiently large contour
        ellipse = cv2.fitEllipse(bc)
        
        center = ellipse[0]
        rotation = ellipse[2]
        width = ellipse[1][0]
        height = ellipse[1][1]
        
        major_axis_lengths.append(height)
        minor_axis_lengths.append(width)        
        
        #draw contour onto a mask
        mask = np.zeros_like(img)
        out = np.zeros_like(img) 
        cv2.drawContours(mask, big_contours, idx, 1, -1)    
        out[mask == 1] = img[mask == 1]
        
        #crop and rotate to contour mask
        rot = rotate_image(out, rotation, center)
        x,y,w,h = cv2.boundingRect(rot)
        rot_crop = rot[y:y+h, x:x+w]
        
        _, rot_crop = cv2.threshold(rot_crop, 0, 1, cv2.THRESH_BINARY) #everything greater than zero as mask       

        floodfill = np.pad(rot_crop.copy(), pad_width = 1, mode= 'constant', constant_values = 0)
        cv2.floodFill(floodfill, None,  (0,0), 2) #fill with 2s - all interior holes will still be zeros
        floodfill[np.where(floodfill == 0)] = 1 #fills in all interior holes
        floodfill = cv2.bitwise_not(floodfill)
        
        mask_vflip = cv2.flip(floodfill, 0)
        mask_hflip = cv2.flip(floodfill, 1)
        
        #compare overlap of horizontal/vertical flips
        vflip_or_floodfill = np.sum(cv2.bitwise_or(mask_vflip, floodfill))
        hflip_or_floodfill = np.sum(cv2.bitwise_or(mask_hflip, floodfill)) 
        if vflip_or_floodfill > 0:
            asymmetry_v = np.sum(cv2.bitwise_and(mask_vflip, floodfill))/vflip_or_floodfill
        else:
            asymmetry_v = 0
            
        if hflip_or_floodfill > 0: # no overlap
            asymmetry_h = np.sum(cv2.bitwise_and(mask_hflip, floodfill))/hflip_or_floodfill
        else:
            asymmetry_h = 0
        
        asymmetry_vs.append(asymmetry_v)
        asymmetry_hs.append(asymmetry_h)
        
    return major_axis_lengths, minor_axis_lengths, asymmetry_vs, asymmetry_hs

def get_asymmetry(img):
    contours,hierarchy = cv2.findContours(img, 1, cv2.CHAIN_APPROX_NONE)
    
    contour_areas = np.array([cv2.contourArea(c) for c in contours])
    if np.sum(contour_areas > 0):
        proportional_contour_areas = contour_areas/np.sum(contour_areas)
    else:
        print('No contours found, returning all-zero asymmetry')
        return [0,0,0,0,0,0,0,0] 
    #extract contours of interest as all contours with >10% of contour area
    big_indices = np.where(proportional_contour_areas > 0.1)
    big_contours = np.array(contours)[big_indices] # select contours that have > 10% of the total area of all contours
    big_contour_areas = np.array(contour_areas)[big_indices]

    perimeters = [cv2.arcLength(c, True) for c in big_contours]
    
    major_length, minor_length, asymmetry_vs, asymmetry_hs = get_asymmetry_from_contours(img, big_contours)
       
    a_p_ratio = []
    compactness_index = []
    p_times_a = []
    
    for a, p in zip(big_contour_areas, perimeters):
        if p == 0:
            p = 1
            print('0 perimeter')
        a_p_ratio.append(a/p)
        compactness_index.append((4*np.pi*a)/p**2)
        p_times_a.append(p*a)
        
    d1_primes = [np.sqrt((4*a)/np.pi) for a in big_contour_areas]
    d1_double_primes = [(D+d)/2 for D, d in zip(major_length, minor_length)]
    d1 = np.mean([(d1_prime + d1_double_prime)/2 for d1_prime, d1_double_prime in zip(d1_primes, d1_double_primes)])
    d2 = np.mean([np.abs(D-d) for D, d in zip(major_length, minor_length)])
    features = [np.mean(a_p_ratio), np.mean(compactness_index), np.mean(p_times_a), np.mean(asymmetry_vs), np.mean(asymmetry_hs), len(big_contour_areas),  d1, d2]
    return features

def extract_features(imgs,masks, filenames, train):
    X = np.empty((imgs.shape[0], 785))
    kernel = np.ones((1,1),np.uint8)
    
    #load df for filename/demographic ingo
    if train == 'train':
        df = train_df
    elif train == 'valid':
        df = val_df
    else:
        df = test_info 
        
    i = 0   
    for mask, img, filename in zip(masks, imgs, filenames):
        #turn BCDU mask into binary mask
        ret, thresh = cv2.threshold(mask, np.mean(mask.flatten())*1.1, 255, 0)
        mask = cv2.dilate(thresh,kernel,iterations=10)

        if np.sum(mask)==0: # prevent masking with all 0 mask -> segmentation failed
            mask=np.ones(mask.shape)
            asymmetry = [0,0,0,0,0,1, 0 , 0]
            print("Segmentation failed")
            hist = np.array([cv2.calcHist([img], [j], None, [256],  [0,256]) for j in range(3)]).flatten()

        else:
            mask =np.uint8(mask.reshape(256,256))
            asymmetry = get_asymmetry(mask)
            #channel-wise histogram of pixel values
            hist = np.array([cv2.calcHist([img], [j], mask, [256],  [0,256]) for j in range(3)]).flatten()
    
        demographics  = df.loc[df['image_name']== filename].values[0][-9:]
        features = list(hist)+list(demographics)+asymmetry
        X[i] = features
        i+=1
        
    return X

def remove_hairs(image):
    # convert image to grayScale
    image = np.uint8(image)
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # kernel for morphologyEx
    kernel = cv2.getStructuringElement(1,(17,17))
    # apply MORPH_BLACKHAT to grayScale image
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    # apply thresholding to blackhat
    _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    # inpaint with original image and threshold image
    final_image = cv2.inpaint(image,threshold,1,cv2.INPAINT_TELEA)
    return final_image

# %% [code]
t1 = time.time()
epochs = 100
patience = 25

best_val_auc = 0
epochs_since_best =0
history_across_epochs = {}
validation_history = {}
for e in range(epochs):
    print('Epoch', e)
    
    #TRAIN
    batches = 0 
    n_train_iter = len(train_df)//seg_train_generator.batch_size
    bh_loss = []
    bh_auc = []
    bh_prec = []
    bh_rec = []
    bh_acc = []
    for X_train, y_train in seg_train_generator: # generator sampled images
        if batches >= n_train_iter:
            break
        X_train = np.array([remove_hairs(x) for x in X_train])
        
        idx = (seg_train_generator.batch_index - 1) * seg_train_generator.batch_size
        filenames= seg_train_generator.filenames[idx : idx + seg_train_generator.batch_size]
        #process masks to get just melanoma from images       
        segmented_masks_train = segmentation_model.predict(X_train)
        X_train = extract_features(X_train, segmented_masks_train, filenames, train = 'train')
        
        history = prediction_model.fit(X_train, y_train, verbose = 0, batch_size = X_train.shape[0]) 
        
        #collect training metriics
        bh_loss.append(history.history['loss'])
        bh_auc.append(history.history['auc'])
        bh_acc.append(history.history['accuracy'])
        bh_prec.append(history.history['precision'])
        bh_rec.append(history.history['recall'])      
        
        batches +=1
        
    history_across_epochs[e] = {'loss':np.nanmean(bh_loss), 'auc':np.nanmean(bh_auc), 'acc':np.nanmean(bh_acc), 
                     'prec':np.nanmean(bh_prec),'rec':np.nanmean(bh_rec)}
    print('Training Loss:', history_across_epochs[e]['loss'], 'Training AUC', history_across_epochs[e]['auc'], round(time.time()-t1, 2))
    t1 = time.time()
    
    #VALIDATE AND ASSESS

    
    valid_batches = 0

    loss_fnctn = BinaryCrossentropy()
    n_val_iter = len(val_df)//seg_valid_generator.batch_size
    
    all_pred = []
    
    all_true = []
    for X_valid, y_valid in seg_valid_generator: # generator sampled images
        if valid_batches >= n_val_iter:
            break
        #process masks to get just melanoma from images 
        X_valid =  np.array([remove_hairs(x) for x in X_valid])

        segmented_masks_valid = segmentation_model.predict(X_valid)
        
        idx = (seg_valid_generator.batch_index - 1) * seg_valid_generator.batch_size
        filenames= seg_valid_generator.filenames[idx : idx + seg_valid_generator.batch_size]
        
        X_valid= extract_features(X_valid, segmented_masks_valid, filenames, train='valid')
        
        y_pred = list(prediction_model.predict(X_valid).flatten())
        all_pred += y_pred

        all_true += list(y_valid)
        
        valid_batches +=1

        collect()
        
    loss = loss_fnctn(FloatTensor(all_true), FloatTensor(all_pred))
    
    print(np.sum(all_true), np.sum(all_pred))
    if np.sum(all_true) > 0 and np.sum(all_pred) > 0:
        auc = roc_auc_score(all_true, all_pred)
    else:
        auc = np.nan
 
    y_pred_accuracy = np.array([0.0  if i <0.5 else 1.0 for i in all_pred])
    accuracy = np.sum(all_true == y_pred_accuracy)/len(y_pred_accuracy) 
        
    print('Val loss:', loss, 'Val AUC:',  auc, 'Val Accuracy:', accuracy)
    
    validation_history[e] = {'loss':loss, 'auc':auc, 'acc':accuracy}
    
    if auc > best_val_auc:
        print('Validation AUC improved from best value. Saving to best_model.h5')
        best_val_auc = auc
        prediction_model.save('best_model.h5')
        epochs_since_best = 0
    else:
        epochs_since_best +=1 
        print('Validation AUC did not improve from', best_val_auc)
        
        
    if epochs_since_best > patience:
        print('No improvement in ', patience,  'epochs. Quitting.')
        break 
    
    collect()

print(validation_history)
df = pd.DataFrame(validation_history)
print(df)
df.to_csv('Validationhistory.csv')

print(history_across_epochs)
df2 = pd.DataFrame(history_across_epochs)
print(df2)
df2.to_csv('Trainhistory.csv')


0    117
1    117
Name: target, dtype: int64
0    467
1    467
Name: target, dtype: int64
Found 934 non-validated image filenames belonging to 2 classes.
Found 234 non-validated image filenames belonging to 2 classes.
Found 10982 non-validated image filenames.
Epoch 0


NameError: name 'collect' is not defined